In [4]:
from collections import deque
import math
import time
import os
import csv
import numpy as np

def read_maps(size, n_sp, n_tar, id):
    map_start = []
    map_end = []
    T = []
    with open(f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\start\\graph_start_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt", "r") as file_start:
        T_ub = int(file_start.readline())
        for line in file_start.readlines():
            # map(function/type, iterable_object)，把測資用 map 轉整數，然後轉成 list
            map_start += [list(map(int, line.split(',')))]
    
    with open(f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\end\\graph_end_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt", "r") as file_end:
        T_ub = int(file_end.readline())
        for line in file_end.readlines():
            map_end += [list(map(int, line.split(',')))]

    return map_start, map_end

# Function to print the 3x3 grid
def print_grid(state):
    for row in state:
        print(' '.join(str(x) if x >= 0 else 'X' for x in row))
    print()

def print_queue(queue):
    print('deque(')
    for i in queue:
        print(f'\t{i}')
    print(')')

# postprocess: merge the moves
def merge_moves(path):
    path = list(reversed(path))     #gotta reverse the list
    merged_path = []
    merged_path.append(path[0])     #store the initial state first
 
    # for k in range(len(path)):
    #     for i in range(len(path[k])):
    #         for j in range(len(path[k])):
    #             if path[k][i][j] != path[k+1][i][j]:
    #                 # Calculate the difference in positions
    #                 diff_i = i - path[k+1].index(path[k][i][j])
    #                 diff_j = j - path[k+1][j].index(path[k][i][j])
    #                 # Check if the difference is within 1 grid
    #                 if abs(diff_i) + abs(diff_j) <= 1:
    #                     # If within 1 grid, it's a valid merge
    #                     merged_path.append(path[k+1][i][j])
    #                 else:
    #                     # If not within 1 grid, it's not a valid merge
    #                     pass
    # if 

    #     can_merge(path[i], path[i+1])
    pass

def can_merge(path1, path2):
    pass

# def merge_moves(moves):
#     merged = []
#     current_move = None

#     for move in moves:
#         if not current_move:
#             current_move = move
#         elif can_merge(current_move, move):
#             current_move = merge_single_move(current_move, move)
#         else:
#             merged.append(current_move)
#             current_move = move

#     if current_move:
#         merged.append(current_move)

#     return merged

# def can_merge(move1, move2):
#     # 检查两个移动是否可以合并
#     return all(
#         abs(m1) + abs(m2) <= 1 and not (m1 * m2 < 0)
#         for m1, m2 in zip(move1, move2)
#     )

# def merge_single_move(move1, move2):
#     # 合并两个移动
#     return tuple(m1 + m2 for m1, m2 in zip(move1, move2))



# BFS function to find the shortest number of moves and print each step
def bfs_min_moves(start, target):
    timelimit = 500
    start_time = time.time()
    size = (len(start), len(start[0]))
    start_state = tuple(tuple(row) for row in start)
    target_state = tuple(tuple(row) for row in target)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    visited = set()
    visited.add(start_state)
    queue = deque([(start_state, 0, None)])  # (state, distance, parent)
    parent_map = {start_state: None}   # {current_state: previous_state}
    
    while queue:
        current_state, depth, parent = queue.popleft()

        # all() 用來檢查一個可迭代對象中的所有元素是否都為 True。
        # if current_state == target_state:
        if all(
            current_state[i][j] == target_state[i][j] for i in range(size[0]) for j in range(size[1]) if target_state[i][j] > 0  # 只考慮目標狀態中大於 0 的數字
        ):
            # If we've reached the goal, reconstruct the path
            path = []
            while current_state is not None:
                path.append(current_state)
                current_state = parent_map[current_state]        
                
            merge_moves(path)
            end_time = time.time()
            duration = end_time - start_time
            path = list(reversed(path))
            return depth, duration, path
        
        empty_positions = [(i, j) for i in range(size[0]) for j in range(size[1]) if current_state[i][j] == 0]

        for empty_pos in empty_positions:
            x, y = empty_pos
            for direction in directions:
                new_x, new_y = x + direction[0], y + direction[1]
                if 0 <= new_x < size[0] and 0 <= new_y < size[1] and current_state[new_x][new_y] != 0:
                    # Make a move (swap non-zero with zero)
                    new_state = [list(row) for row in current_state]  # Copy current state
                    new_state[x][y], new_state[new_x][new_y] = new_state[new_x][new_y], new_state[x][y]
                    new_state_tuple = tuple(tuple(row) for row in new_state)
                    
                    # If this state hasn't been visited, enqueue it
                    if new_state_tuple not in visited:
                        visited.add(new_state_tuple)
                        parent_map[new_state_tuple] = current_state  # Record parent state
                        queue.append((new_state_tuple, depth + 1, current_state))
        end_time = time.time()
        duration = end_time - start_time
        if duration > timelimit:
            return -1, timelimit, []

    end_time = time.time()
    duration = end_time - start_time
    return np.inf, duration, []  # Return -1 if no solution found

def pairDataDict(size):
    s = size
    datadict = {}
    # [1,2,3,4,5] + [Q1, Q2, Q3] + [n*n-1, n*n//2]
    tar_list = [i+1 for i in range(5) if i+1<= s[0]*s[1]-1]\
                 + [(i+1)*(s[0]*s[1]//4) for i in range(3)]\
                 + [s[0]*s[1]-1, math.floor((s[0]*s[1])/2)]
    tar_list = sorted(list(set(tar_list)-{s[0]*s[1]}))
    for n_tar in tar_list:
        # 空格不同數量：1,2,3,4,5,Quatile, size-target
        spacelist = [i+1 for i in range(5) if i+1 <= s[0]*s[1]-n_tar] \
                  + [(i+1)*((s[0]*s[1])//4) for i in range(3) if (i+1)*((s[0]*s[1])//4) <= s[0]*s[1]-n_tar] \
                  + [s[0]*s[1]-n_tar]
        # spacelist = sorted(list(set(spacelist)-{0,s[0]*s[1]}))
        spacelist = sorted(list(set(spacelist)-{0, s[0]*s[1]}))  # 先把一個空格的也去掉，避免無解
        datadict[n_tar] = spacelist

    return datadict, tar_list

def writefile_bfs(runtime, obj, path, file_path):
    with open(file_path, 'w') as file:
        file.write(f"running time = {runtime}\n")
        file.write(f"optimal value = {obj}\n")
        t = 0
        for state in path:
            file.write('-' * 40 + '\n')
            file.write(f'time period {t}\n')
            for row in state:
                file.write(' '.join(str(x) if x >= 0 else 'X' for x in row))
                file.write('\n')
            t += 1
            

def writeAll(size):
    s = size
    datadict, tar_list = pairDataDict(size)
    # print(datadict)
    # print(tar_list)
    # return
    folder = 'result/algo_bfs'
    os.makedirs(folder, exist_ok=True)
    with open(f'result/algo_bfs/bfs_{size[0]}x{size[1]}.csv', 'w') as f:
        w = csv.writer(f)
        w.writerow(['n_tar', 'n_sp', 'id', 'Runtime', 'Obj'])
        for n_tar, spacelist in datadict.items():
            os.makedirs(f'result_graphform/algo_bfs/{size[0]}x{size[1]}/tar{n_tar}', exist_ok=True)
            for n_sp in spacelist:
                num = 10
                if s[0] == 3 and s[1] == 3:
                    if n_tar == 1 and n_sp == 8: num = 9
                    elif n_sp == 1: num = 9
                for id in range(num):
                    map_start, map_end = read_maps(size, n_sp, n_tar, id)
                    # 處理最佳解
                    obj, runtime, path = bfs_min_moves(map_start, map_end)
                    print(f'tar_{n_tar}, sp_{n_sp}, id_{id}')
                    print("running time = ", runtime)
                    print("optimal value = ", obj)
                    print("-" * 40)
                    # 移動結果寫檔
                    file_path = f'result_graphform\\algo_bfs\\{size[0]}x{size[1]}\\tar{n_tar}\\graph_result_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt'
                    writefile_bfs(runtime, obj, path, file_path)
                    w.writerow([n_tar, n_sp, id, runtime, obj])

size = (3,3)
writeAll(size)

# Run BFS to find the minimum number of moves and print each step
# map_start, map_end = readFile()
# min_moves, duration, path = bfs_min_moves(map_start, map_end)
# print(f"Total moves: {min_moves}")  
# print(f"Total time: {duration}")  
# print(f"Path: ")
# for state in path:
#     print_grid(state)
# print(datadict)
# print(tar_list)

tar_1, sp_1, id_0
running time =  0.0019974708557128906
optimal value =  13
----------------------------------------
tar_1, sp_1, id_1
running time =  0.0
optimal value =  5
----------------------------------------
tar_1, sp_1, id_2
running time =  0.0
optimal value =  1
----------------------------------------
tar_1, sp_1, id_3
running time =  0.000904083251953125
optimal value =  9
----------------------------------------
tar_1, sp_1, id_4
running time =  0.0
optimal value =  1
----------------------------------------
tar_1, sp_1, id_5
running time =  0.0009949207305908203
optimal value =  9
----------------------------------------
tar_1, sp_1, id_6
running time =  0.0
optimal value =  1
----------------------------------------
tar_1, sp_1, id_7
running time =  0.0010058879852294922
optimal value =  9
----------------------------------------
tar_1, sp_1, id_8
running time =  0.0
optimal value =  1
----------------------------------------
tar_1, sp_2, id_0
running time =  0.0059957504